In [1]:
import os
import ee
import geemap
import time
import random
from datetime import datetime, timedelta , date
import ipywidgets as widgets
from IPython.display import display
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
#Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 
Map = geemap.Map()

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

DatePicker(value=None, description='Pick a Star Date')

In [7]:
# give the bands

band_name = widgets.Dropdown(
options=['ndvi','ndwi1','ndwi2','ndwi3','evi','savi','K','C'],
description='Band')

band_name

Dropdown(description='Band', options=('ndvi', 'ndwi1', 'ndwi2', 'ndwi3', 'evi', 'savi', 'K', 'C'), value='ndvi…

In [8]:
# gine the date frequency

frequency =widgets.Dropdown(
    options=['Year','Month'],
    value='Year',
    description='Frequency')

frequency

Dropdown(description='Frequency', options=('Year', 'Month'), value='Year')

In [9]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\USER\Desktop\satellite python code\satellite python code\Geotif', filename='', titl…

In [10]:
# give the scale number
scale = widgets.IntSlider(
    value=1000,
    min=500,
    max=10000,
    step=100,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

scale

IntSlider(value=1000, continuous_update=False, description='Scale:', max=10000, min=500, step=100)

In [11]:
roi = geemap.shp_to_ee(''.join(my_button.files)).geometry()

In [12]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

In [13]:
#Functionalize each parameter

def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.expression(
      '((NIR - RED) / (NIR + RED ))', {
      'NIR' : image.select('nir').multiply(0.0000275).add(-0.2),
      'RED' : image.select('red').multiply(0.0000275).add(-0.2),}).rename("ndvi")
    
    return image.addBands(ndvi);

# get indexes NDWI1
def getNDWI1(image):
    
    ndwi1 = image.expression(
      '((NIR - SWIR1) / (NIR + SWIR1 ))', {
      'NIR' : image.select('nir').multiply(0.0000275).add(-0.2),
      'SWIR1' : image.select('swir1').multiply(0.0000275).add(-0.2),}).rename("ndwi1")
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    

    return image.addBands(ndwi1);

# get indexes NDWI2
def getNDWI2(image):
    
    # Normalized difference vegetation index (NDWI2) (Xnir - Xswir2)/(Xnir + Xswir2)
    ndwi2 = image.expression(
      '((NIR - SWIR2) / (NIR + SWIR2 ))', {
      'NIR' : image.select('nir').multiply(0.0000275).add(-0.2),
      'SWIR2' : image.select('swir2').multiply(0.0000275).add(-0.2),}).rename("ndwi2")
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    

    return image.addBands(ndwi2);


# get indexes NDWI3
def getNDWI3(image):
    
    # Normalized difference vegetation index (NDWI1) (Xgreen - Xnir)/(Xgreen + Xnir)
    ndwi3 = image.expression(
      '((GREEN - NIR) / (GREEN + NIR))', {
      'NIR' : image.select('nir').multiply(0.0000275).add(-0.2),
      'GREEN' : image.select('green').multiply(0.0000275).add(-0.2),}).rename("ndwi3")
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    

    return image.addBands(ndwi3);


def getEVI(image):
    evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('nir').multiply(0.0000275).add(-0.2),
      'RED' : image.select('red').multiply(0.0000275).add(-0.2),
      'BLUE': image.select('blue').multiply(0.0000275).add(-0.2)}).rename('evi');
    return image.addBands(evi);
    

def getSAVI(image):
    savi = image.expression(
     '1.5 * (nir - red) / (nir + red + 0.5)', {
      'nir': image.select('nir').multiply(0.0000275).add(-0.2),
      'red': image.select('red').multiply(0.0000275).add(-0.2)}).rename('savi');
    return image.addBands(savi);



def getK(image):
    
    K = image.expression(
      'temperature',{
      'temperature' : image.select('temperature').multiply(0.00341802).add(149.0)}).rename('K');
    
    return image.addBands(K)

    
    return image.addBands(K2)



def getC(image):
    
    C = image.expression(
      'temperature * 0.00341802 + 149 - 273.15',{
      'temperature' : image.select('temperature')}).rename('C');
    
    return image.addBands(C)

In [14]:
def geotif(time_list):
   
    for i in range(0,len(time_list)):
        star_time = time.time()
        
        dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                             .filterDate(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1)) \
                             .filterBounds(roi) \
                             .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7', 'ST_B10'],
                                     ['ultra blue','blue','green','red','nir','swir1','swir2', 'temperature']) \
                             .map(getNDVI) \
                             .map(getNDWI1) \
                             .map(getNDWI2) \
                             .map(getNDWI3) \
                             .map(getEVI) \
                             .map(getSAVI) \
                             .map(getK) \
                             .map(getC) \
                             .select(band_name.value) \
                             .mean()
        
        image = dataset.clip(roi).unmask()
        filename = os.path.join(os.path.expanduser(folder.selected) + '_{}.tif'.format(time_list[i]))
        geemap.ee_export_image(
            image, filename=filename, scale=scale.value, region=roi, file_per_band=False
                                )

In [15]:
if 'Year' in frequency.value.strip():
    time_list = yearlist(star.value, end.value)
    geotif(time_list)
else:
    pass
if "Month" in frequency.value.strip():
    time_list =monthlist(star.value, end.value)
    geotif(time_list)
else:
    pass

Generating URL ...
Please wait ...
Data downloaded to D:\56789_['2019-01-09', '2019-01-31'].tif
Generating URL ...
Please wait ...
Data downloaded to D:\56789_['2019-02-01', '2019-02-28'].tif
Generating URL ...
Please wait ...
Data downloaded to D:\56789_['2019-03-01', '2019-03-19'].tif
